In [ ]:
# https://www.kaggle.com/maunish/clrp-roberta-svm
# https://www.kaggle.com/anaverageengineer/comlrp-baseline-for-complete-beginners

In [ ]:
import spacy
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge

In [ ]:
RANDOM_STATE = 41
nlp = spacy.load('en_core_web_lg')

In [ ]:
COMPETITION_DATA_PATH = Path('../input/commonlitreadabilityprize')
TRAIN_DATA_PATH = COMPETITION_DATA_PATH / 'train.csv'
TEST_DATA_PATH = COMPETITION_DATA_PATH / 'test.csv'

In [ ]:
train_data = pd.read_csv(TRAIN_DATA_PATH)
test_data = pd.read_csv(TEST_DATA_PATH)
print(f'Length of training data: {len(train_data)}')
print(f'Length of test data: {len(test_data)}')

# Feature Extraction from spacy
All Credits to [Sumit Kumar](https://www.kaggle.com/anaverageengineer) @anaverageengineer https://www.kaggle.com/anaverageengineer/comlrp-baseline-for-complete-beginners

In [ ]:
with nlp.disable_pipes():
    X_train = np.vstack([nlp(text).vector for text in tqdm(train_data['excerpt'])])
    y_train = train_data['target']
    print(f'Shape of Train vectors: {X_train.shape}')

    X_test = np.vstack([nlp(text).vector for text in tqdm(test_data['excerpt'])])
    print(f'Shape of Test vectors: {X_test.shape}')

# Ridge Regressor
From my own notebook: https://www.kaggle.com/vigneshbaskaran/commonlit-a1-pure-scikit-learn

In [ ]:
regressor = Ridge(fit_intercept=True, normalize=False)
scores = cross_val_score(regressor, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
print(f'Average Root mean squared error: {np.abs(np.mean(scores))}')

# Refit model and submit

In [ ]:
regressor = regressor.fit(X_train, y_train)
test_data['target'] = regressor.predict(X_test)
test_data[['id','target']].to_csv('submission.csv', index=False)